# This notebook is based on [this documentation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-models?view=azureml-api-2&tabs=python%2Cuse-local), [this too](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-online-endpoints?view=azureml-api-2&tabs=python#define-the-endpoint), and [this video](https://www.youtube.com/watch?v=luz6Q1R2lkA)

## Downloading and deploying the model

In [5]:
%pip install gdown

import gdown

gdown.download(url="https://drive.google.com/file/d/1pfFmbmzbiPPryt2LEltOvml7HH9SrMVQ/view", output="letter_model.pt", quiet=False, fuzzy=True)

Note: you may need to restart the kernel to use updated packages.


Downloading...
From (uriginal): https://drive.google.com/uc?id=1pfFmbmzbiPPryt2LEltOvml7HH9SrMVQ
From (redirected): https://drive.google.com/uc?id=1pfFmbmzbiPPryt2LEltOvml7HH9SrMVQ&confirm=t&uuid=0dd1adf2-2380-48c6-8c68-f9773e8ce5d7
To: /mnt/batch/tasks/shared/LS_root/mounts/clusters/tempcomputeocr/code/Users/eng-youssef.victor1722/letter_model.pt
100%|██████████| 546M/546M [00:06<00:00, 78.8MB/s] 


'letter_model.pt'

In [3]:
!curl https://raw.githubusercontent.com/SeniorProject-2023/ocr-app/main/src/Backend/InferenceServer/word_model.pt -o word_model.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 85.5M  100 85.5M    0     0  60.4M      0  0:00:01  0:00:01 --:--:-- 60.4M


In [18]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

subscription_id = "f4f75b61-8d75-4263-8b2a-bcfac715f231"
resource_group = "ocrapp"
workspace = "ocrworkspace"

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

In [8]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="letter_model.pt",
    type=AssetTypes.CUSTOM_MODEL,
    name="letter_model",
    description="Arabic OCR Letter Model",
)
ml_client.models.create_or_update(file_model)

Uploading letter_model.pt (< 1 MB): 482MB [00:03, 146MB/s]                             




Model({'job_name': None, 'is_anonymous': False, 'auto_increment_version': False, 'name': 'letter_model', 'description': 'Arabic OCR Letter Model', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/f4f75b61-8d75-4263-8b2a-bcfac715f231/resourceGroups/ocrapp/providers/Microsoft.MachineLearningServices/workspaces/ocrworkspace/models/letter_model/versions/1', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/tempcomputeocr/code/Users/eng-youssef.victor1722', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f98f2504a00>, 'serialize': <msrest.serialization.Serializer object at 0x7f98e9cae1a0>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/f4f75b61-8d75-4263-8b2a-bcfac715f231/resourceGroups/ocrapp/workspaces/ocrworkspace/datastores/workspaceblobstore/paths/LocalUpload/1aec4cef497641942541d9a5b92faaf0/letter_model.pt', 'datastore': None, 'utc_time_created': None, 'flavors

## Deploying real-time endpoint

In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))


Ready to use Azure ML 1.49.0 to work with ocrworkspace


In [2]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')


letter_model version: 2
	 azureml.datastoreId : /subscriptions/f4f75b61-8d75-4263-8b2a-bcfac715f231/resourceGroups/ocrapp/providers/Microsoft.MachineLearningServices/workspaces/ocrworkspace/datastores/workspaceblobstore


letter_model version: 1
	 azureml.datastoreId : /subscriptions/f4f75b61-8d75-4263-8b2a-bcfac715f231/resourceGroups/ocrapp/providers/Microsoft.MachineLearningServices/workspaces/ocrworkspace/datastores/workspaceblobstore




In [3]:
model = ws.models['letter_model']
print(model.name, 'version', model.version)


letter_model version 2


In [11]:
import os

# Create a folder for the deployment files
deployment_folder = './ocr_deployment'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score.py'
script_path = os.path.join(deployment_folder, script_file)

env_file = 'conda.yml'
env_path = os.path.join(deployment_folder, env_file)


./ocr_deployment folder created.


In [12]:
%%writefile $script_path
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'letter_model.pt')
    # TODO:

# Called when a request is received
def run(raw_data):
    return {"Hello": "Hellloooo"}


Overwriting ./ocr_deployment/score.py


In [28]:
%%writefile $env_path
name: inference_env
dependencies:
- python = 3.10.11
- azureml-inference-server-http


Overwriting ./ocr_deployment/conda.yml


In [51]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

# Define an endpoint name
#endpoint_name = "arabic-y-ocr2023"

# Example way to define a random name
import datetime

endpoint_name = "endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="Arabic OCR",
    auth_mode="key"
)

In [52]:
model = Model(path="./letter_model.pt")
env = Environment(
    conda_file=deployment_folder + "/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code=deployment_folder, scoring_script=script_file
    ),
    instance_type="Standard_E4s_v3",
    instance_count=8,
)

In [53]:
ml_client.online_endpoints.begin_create_or_update(endpoint)


In [55]:
ml_client.online_deployments.begin_create_or_update(blue_deployment)

Check: endpoint endpt-06231711704994 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


...........

In [37]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint)

HttpResponseError: (BadRequest) The request is invalid.
Code: BadRequest
Message: The request is invalid.
Exception Details:	(InferencingClientCallFailed) {{"errors":{{"DeploymentWeights":["The deployments [blue] on which you are trying to set the traffic are in a failed state and don't have ingress. Please fix the deployments and try again."]}},"title":"One or more validation errors occurred."}}
	Code: InferencingClientCallFailed
	Message: {{"errors":{{"DeploymentWeights":["The deployments [blue] on which you are trying to set the traffic are in a failed state and don't have ingress. Please fix the deployments and try again."]}},"title":"One or more validation errors occurred."}}
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "f1164af52782e22401516e4d3b24f9ca",
        "request": "57c0f117564b4ef8"
    }
}Type: Environment
Info: {
    "value": "eastus2"
}Type: Location
Info: {
    "val